In [1]:
import numpy as np

In [9]:
# physical parameters

M_p = 1 # total mass of planet in Earth masses
a = 1 # semi-major axis in AU
Alb = 0.1 # planetary albedo
S_0 = 1361 # instellation at 1 AU in W m^-2
Ur = 1 # Urey ratio
F_c = 1220e3 / 6371e3 # core fraction in terms of radius

T_0_m = 1800 # initial mantle temperature in K
T_0_c = 2500 # initial core temperature in K (think u don't need this yet)
rho_m = 4500 # density of mantle in kg m^-3
rho_c = 7800 # density of core in kg m^-3 (in Bryson+ 2015)

nu_m = 1e20 # Mantle visc in Pa s
Ra_c = 660 # critical Rayleigh number (in Driscoll & Bercovici 2014)
C_pm = 800 # specific heat capacity of silicate mantle in J kg−1 K−1 (Elkins-Tanton+ 2011)
alpha_m = 2e-5 # thermal expansivity of silicate mantle in K^-1
k_m = 4 # thermal conductivity of silicate mantle in W m^−1 K^−1
C_pc = 850 # specific heat capacity of iron core in J kg^−1 K^−1 (Elkins-Tanton+ 2011)

X_K = 250 # initial abundance of K in wt ppm (in Treatise on Geophysics, think these are by weight but double check)
X_U = 2e-2# initial abundane of U in wt ppm ""
X_Th = 7e-2 # initial abundance of Th in wt ppm ""
K_0 = 0.0117e-2 # ratio of 40-K to total K at time 0 (in Treatise on Geophysics)
U_0_235 = 0.0072 # ratio of 235-U to total U at time 0 (in Treatise on Geophysics)
U_0_238 = 0.9927 # ratio of 238-U to total U at time 0 (in Treatise on Geophysics)
Th_0 = 1 # ratio of 232-Th to total Th at time 0 (in Treatise on Geophysics)

sb = 5.670374419e-8 # Stefen-Boltzmann constant in W m^−2 K^−4

# numerical parameters
tau_f = 4.5e9 # total time in years
dtau = 100e6 # time-step in years

M_E = 5.972e24 # earth mass in kg
years2sec = 31557600
params = dict(M_p=M_p*M_E, a=a, Alb=Alb, F_c=F_c, rho_m=rho_m, rho_c=rho_c, 
              Ra_c=Ra_c, C_pm=C_pm, alpha_m=alpha_m, k_m=k_m, Ur=Ur, nu_m=nu_m,
              X_K=X_K, X_U=X_U, X_Th=X_Th, K_0=K_0, U_0_235=U_0_235, U_0_238=U_0_238, Th_0=Th_0,
              sb=sb,
              tau_f=tau_f*years2sec, dtau=dtau*years2sec)
init = dict(T_0_m=T_0_m)

In [ ]:
""" 0-D steady state model, homogenous interior """

M_E = 5.972e24 # earth mass in kg
years2sec = 31557600

def q_star(S_0, Alb, a):
    return S_0*(1-Alb)/(4*a**2)

def H_rad(tau, H_0, X_0, el_0, t_half):
    H = 0
    for ii in np.size(H_0):
        H += (H_0[ii]*X_0[ii]*el_0[ii]*np.exp(np.log(2)*tau/t_half[ii]))
    return H

def q_out_ss(a, M_p, tau, S_0=1361, rho_m=4500,
             H_0 = [],
             X_0 = [],
             el_0 = [],
             t_half = []
             ):
    q_bb = q_star(S_0, Alb, a) + rho_m/M_p*H_rad(tau, H_0, X_0, el_0, t_half)
    return q_bb

M_p_list = [0.5, 1, 1.5, 2, 2.5] # Earth masses
a_list = [0.5, 1, 1.5, 2, 2.5, 3] # AU
age_list = [1, 2, 3, 4] # Gyr

for M_p in M_p_list:
    for a in a_list:
        for age in age_list:
            q_out = q_out_ss(a, M_p, age)

        

In [6]:
def secular_cooling(dtau, M, C_v, q_r, H, R_outer, R_inner=0):
    """Secular cooling equation. Ignores adiabatic heating from contraction, external energy transfers, 
    work by atmospheric pressure. Assumes spatially uniform heat flux and heat production. 
    Might assume this is 0 (Urey ratio of unity)
    
    Parameters
    ----------
    dtau : float
        Time step
    M : float
        Mass of reservoir in kg
    C_v : float 
        Specific heat by volume  
    q_r : float 
        Heat flux in W m**-2 
    H : float
        Volumetric internal heat production in W m**-3
    R_outer : float
        Outer radius of reservoir in m
    R_inner : float (optional)
        Inner radius of reservoir in m. Defaults to 0
        

    Returns
    -------
    dT : float
        Surface temperature change over time step in K
    """
    
    SA = 4*np.pi*R**2 # outer surface area of reservoir
    V = 4/3*np.pi*(R_outer**3 - R_inner**3) # volume of reservoir
    dT = (-q_r*SA + H*V) / (M*C_v) * dtau
    return dT

def thermal_diffusivity(k, rho, C_p):
    """
    Calculate thermal diffusivity
    
    Parameters
    ----------
    k : Thermal conductivity
    C_p : Specific heat capacity in J K^-1 kg^-1
    rho : density in kg m^-3
    """
    return k/(rho*C_p)
    
def rad_flux(tau, K_0, X_K, U_0_235, U_0_238, X_U, Th_0, X_Th):
    """Calculate volumetric radiative heating from K-U-Th decay
    
    Parameters
    ----------
    tau : float
        Time elapsed
    X_K : float
        Abundance of K accreted in wt. %
    K_0 : float
        Ratio of 40-K to 39-K at tau = 0
    etc.
        
    Returns
    -------
    h_V : float
        Radiative heat production in W kg^-1 
    """
    
    # Half-lives in years from Dye (2012) in Treatise on Geophys
    t_40K_half = 1.26e9 
    t_235U_half = 7.04e8 
    t_238U_half = 4.46e9
    t_232Th_half = 1.4e10
    
    # Heating rates of radioisotopes at tau=0 in W kg^-1 from Dye (2012) in Treatise on Geophys
    h_40K_0 = 28.47e-6
    h_235U_0 = 568.47e-6
    h_238U_0 = 95.13e-6
    h_232Th_0 = 26.3e-6
    
    # TODO: not sure if U-Th system can be added linearly?
    # TODO: does it matter that 40-K has two decay processes? account separately? different heating
    h_40K = h_40K_0 * K_0 * X_K * np.exp(np.log(2)*(tau/t_40K_half))
    h_235U = h_235U_0 * U_0_235 * X_U * np.exp(np.log(2)*(tau/t_235U_half))
    h_238U = h_238U_0 * U_0_238 * X_U * np.exp(np.log(2)*(tau/t_238U_half))
    h_232Th = h_232Th_0 * Th_0 * X_Th * np.exp(np.log(2)*(tau/t_232Th_half))
    h = h_40K + h_235U + h_238U + h_232Th
    return h 

def ubl_cond(A, k_m, T_s, T_m, d_lid):
    """Heat conduction through upper boundary layer, this sets convective cooling
    
    Parameters
    ----------
    A : float
        Surface area in m^2 
    k_m : float
        Thermal conductivity of upper mantle
    T_s : float
        Surface temperature in K
    T_m : float
        Mantle temperature in K
    d_lid : float
        Crust thickness in m
    
    Returns
    ------
    q : float
        Heat flux in W m^-2  [OR W m^-1???]
    """
    return A*k_m*(T_m - T_s)/d_lid
    
def CMB_flux(q_cond, q_rad, Ur=1): 
    """Calculate heat flux across core-mantle boundary, assuming a fixed Urey ratio
    
    Parameters
    ----------
    Ur : Urey ratio
    q_cond : Conductive flux across upper thermal bdy layer
    q_rad : Radiogenic heating flux (volumetric) in mantle
    """
    q_CMB = (q_cond-q_rad)/Ur # assume total equilibrium

    return q_CMB # W m**-2

def lid_thickness(T_s, T_m, Ra_c, nu_m, kappa_m, alpha_m, g):
    """Thickness of uppper mantle thermal boundary layer 
    
    Parameters
    ----------
    T_s : float
        Surface temperature
    Ra_c : float
        Critical Rayleigh number
    alpha_m : float 
        Thermal expansivity in
    kappa : float 
        Thermal diffusivity in
    nu_m : float
        Mantle viscosity in
    g : float
        Acceleration due to gravity in m s^-2
    
        
    Returns
    -------
    d : float
        Lid thickness in m
    """
    
    # Set Rayleigh number at critical value
    d = (Ra_c * nu_m*kappa_m/(alpha_m*g*(T_m - T_s)))**(1/3)
    return d

def surf_temp_radeq(a, Alb, S_0):
    """Calculate surface temperature assuming no greenhouse effect and only radiative equilibrium

    Parameters
    ----------
    a : float
        Semi-major axis in AU
    S : float (optional)
        Solar constant at 1 AU and 4.5 Gyr in W m^-2. Defaults to solar value    
    Al : float 
        Albedo. Defaults to 0.1 (arbitrarily assumed)
        

    Returns
    -------
    T_surf : float
        Surface temperature in K
    """
    
    # TODO: solar luminosity increases with age, e.g. ~30% over 2 Gyr
    T_surf = ( (((1 - Alb)*S / (4*a**2))) / sb )**0.25
    return T_surf

def planet_radius(M_p, F_c, rho_m, rho_c):
    """Calculate radius of planet given total mass, assume lid has the same density as mantle
    
    Parameters
    ----------
    M_p : float
        Mass of planet
    rho_m : float
        Mantle density
    F_c : float
        Core fraction in terms of radius
    rho_c : float
        Core density
    """
    
    R_p = ( 3*M_p/(4*np.pi) / (rho_m - F_c**3*rho_m + F_c**3*rho_c) )**(1/3)
    return R_p

def gravity_sfc(M_p, R_p):
    """Calculate acceleration due to gravity in m s^-2"""
    return 6.674e-11*M_p/R_p**2



In [ ]:
# def start(**init, **params):
#     T_m_list = []
#     T_s_list = []
#     T_m = T_m_0 # initial mantle temp
#     T_s = surf_temp_radeq(a, Alb, S_0) # initial surface temp
#     for ii in range(0, tau_f, np.floor(dtau/tau_f)):
#         T_m_list.append(T_m)
#         T_s_list.append(T_s)
#         # advance model
#         T_m_1, T_s_1 = step(ii*dtau, T_m, T_s, **params)
#         T_m = T_m_1
#         T_s_1 = T_s_1
        

def solve_ss(age, **params):
    """Advance model"""
    
    # 0. Update physical parameters
    R_p = planet_radius(M_p, F_c, rho_m, rho_c)
    SA_p = 4*np.pi*R_p**2
    V_m = 4/3*np.pi*((R_p)**3 - (F_c*R_p)**3) #  4/3*np.pi*((R_p - d_lid)**3 - (F_c*R_p)**3)
    g = gravity_sfc(M_p, R_p)
    kappa_m = thermal_diffusivity(k_m, rho_m, C_pm)
    R_c = R_p*F_c 
    
    
#     # 1. Update lid thickness 
#     d_lid = lid_thickness(T_s, T_m, Ra_c, kappa_m, alpha_m, g)
    
    # 2. Calculate internal heating --- only radiogenic
    H_rad = rad_flux(tau=age, K_0, X_K, U_0_235, U_0_238, X_U, Th_0, X_Th)
    q_m_internal = q_rad/V_m # convert to volumetric
    
    # Calculate T_s assuming negligible geothermal component
    T_s = surf_temp_radeq(a, Alb, S_0)
    
    # 3. Update fluxes
    q_m_out = ubl_cond(SA_p, k_m, T_s, T_m, d_lid) 
    q_m_in = CMB_flux(q_m_out, q_m_internal) # assume Urey ratio 1

    # 4. Solve for steady state temperatures
    T_s = # assume conductive flux has minimal effect if no roots?
    T_m = 
 
    
    # Calculate new mantle temperature
    #delta_T_m = (-q_m_out*SA_p + H_m*rho_m*V_m + q_m_in*SA_c) / (rho_m*C_pm*V_m) * dtau
    
    # Calculate new sfc temperature
    #delta_T_s = 
    


In [ ]:
def Ra(nu, alpha, rho, g, T_m, T_s, d, kappa, ):
    """Calculate Rayleigh number of convecting material

    Parameters
    ----------
    nu : float
        Viscosity
    alpha : float 
        Thermal expansivity
    rho : float
        Density
    kappa : float 
        Thermal diffusivity 
    d : float
        Thickness of convecting layer
    """
    
    g = 9.81 # Acceleration due to gravity
    
    return alpha*rho*g*(T_m - T_s)*d**3 / (kappa*nu)

def viscosity(T, nu_0, A_v):
    """Calculate Rayleigh number of convecting material

    Parameters
    ----------
    T : float 
        Temperature in K
    nu_0 : float
        Reference viscosity
    A_v : float
        Activation energy
    """
    
    R_g = # gas constant
    nu = nu_0 * np.exp(A_v)
    
    # isoviscous for now
    return nu_0

